In [4]:
! nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [0]:
!pip install -U -q PyDrive

from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print('success!')

success!


In [2]:
from progressbar import ProgressBar, Percentage, Bar
import torch, pickle, os, sys, random, time, math, copy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# german = drive.CreateFile({'id': '1I0_go5RhzEg2CkbpZgx8a6h4hA9qNeyY'})
# german.GetContentFile('./german_no_pad_sorted.pickle') 
# with open('./german_no_pad_sorted.pickle', 'rb') as f_in:
#     german = pickle.load(f_in)
    
# english = drive.CreateFile({'id': '1Tb0aYVfkj3YyXXTxWRkvDO7T__ovng1Q'})
# english.GetContentFile('./english_no_pad_sorted.pickle') 
# with open('./english_no_pad_sorted.pickle', 'rb') as f_in:
#     english = pickle.load(f_in)    
    
german = drive.CreateFile({'id': '10epaM5VzSskc0xkriD0YN_a0Mg3-o1bg'})
german.GetContentFile('./german_no_pad_sorted_5k.pickle') 
with open('./german_no_pad_sorted_5k.pickle', 'rb') as f_in:
    german = pickle.load(f_in)

english = drive.CreateFile({'id': '15P0N9sLAIPyxF0Uhgx-01P2s-xOLp6xb'})
english.GetContentFile('./english_no_pad_sorted_5k.pickle') 
with open('./english_no_pad_sorted_5k.pickle', 'rb') as f_in:
    english = pickle.load(f_in)


training_data = [[german['train'][i], english['train'][i]] for i in range(len(german['train']))]
validation_data = [[german['dev'][i], english['dev'][i]] for i in range(len(german['dev']))]

max_len_train = len(max(german['train'], key=len))
max_len_valid = len(max(german['dev'], key=len))

print(max_len_train, max_len_valid)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
def to_padded_tensor(batch):
    max_len_src = max([len(sent['source']) for sent in batch])  
    max_len_trg = max([len(sent['target']) for sent in batch])

    for sent in batch:
        dif_src = max_len_src - len(sent['source'])
        dif_trg = max_len_trg - len(sent['target'])

        if dif_src > 0:
            pad_list_src = [0 for d in range(dif_src)]
            sent['source'].extend(pad_list_src)

        if dif_trg > 0:
            pad_list_trg = [0 for d in range(dif_trg)]
            sent['target'].extend(pad_list_trg)       
    
    source_sent_len = max_len_src
    target_sent_len = max_len_trg
    
    batch_size = len(batch)
    
    source = torch.empty((source_sent_len, batch_size)).long().cpu()    
    target = torch.empty((target_sent_len, batch_size)).long().cpu()
    
#     print(batch[0]['target'])
    
    for i in range(len(batch)):
        source[:,i] = torch.tensor(batch[i]['source'])
        target[:,i] = torch.tensor(batch[i]['target'])
        
    padded_tensor = {"source": source.to(device),
                    "target": target.to(device)}
    
    return padded_tensor

In [0]:
def bake_batches(de, en, batch_size=1300, min_len=3, max_len=768, bucket_step=3):
    german = copy.deepcopy(de)
    english = copy.deepcopy(en)
    
    buckets = [[] for i in range(0, max_len, bucket_step)]
    bucket_lengths = [0 for i in buckets]
    batches = []
    
    # For every sentence in the dataset, find its corresponding bucket and put it in there, once the bucket
    # hits the batch size, ship it off to the batches list
    for i in range(len(german)):
        sent2sent = {"source": german[i],
                     "target": english[i]}
        
        # calculate the index of the buckets to put the sentence into, = len(Sentence) // Bucket_step - 1
        idx = len(sent2sent['source'])//bucket_step - 1
        
        if bucket_lengths[idx] + len(sent2sent['source']) > batch_size:
            batches.append(to_padded_tensor(buckets[idx][:]))
            del buckets[idx][:]
            buckets[idx].append(sent2sent)
            bucket_lengths[idx] = len(sent2sent['source'])
        else:
            buckets[idx].append(sent2sent)
            bucket_lengths[idx] += len(sent2sent['source'])
            

    # for any remaining buckets that did not get sent off, send them off to batches
    for b in buckets:
        if b: # if the list has any value in it
            batches.append(to_padded_tensor(b[:]))
            del b[:]
    
    return batches

In [6]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src sent len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src sent len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src sent len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [7]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat encoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src sent len, dec hid dim]
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src sent len, dec hid dim]
        
        energy = energy.permute(0, 2, 1)
        
        #energy = [batch size, dec hid dim, src sent len]
        
        #v = [dec hid dim]
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)
        
        #v = [batch size, 1, dec hid dim]
                
        attention = torch.bmm(v, energy).squeeze(1)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [8]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [sent len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #sent len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        output = self.out(torch.cat((output, weighted, embedded), dim = 1))
        
        #output = [bsz, output dim]
        
        return output, hidden.squeeze(0)

In [9]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        output = trg[0,:]
        
        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs

In [10]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [11]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512)
    (out): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5)
  )
)

In [14]:
optimizer = optim.Adam(model.parameters())

In [15]:
criterion = nn.CrossEntropyLoss(ignore_index = 0)

In [0]:
def train(model, batches, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, b in enumerate(batches):
        if i%100 == 0: 
            print( "batch:", i, ",", torch.cuda.memory_allocated(device)/1e6, "MB used") 

        source = b['source']
        target = b['target']
        
        optimizer.zero_grad()
        
        output = model(source, target)
        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, output.shape[-1])
        target = target[1:].view(-1)
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, target)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(batches)

In [0]:
def evaluate(model, batches, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, b in enumerate(batches):
            
            source = b['source']
            target = b['target']

            output = model(source, target, 0) #turn off teacher forcing

            #trg = [trg sent len, batch size]
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            target = target[1:].view(-1)

            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, target)

            epoch_loss += loss.item()
        
    return epoch_loss / len(batches)

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    random.shuffle(training_data)
    random.shuffle(validation_data)

    de_shuffled_td = [td[0] for td in training_data]
    en_shuffled_td = [td[1] for td in training_data]
    
    de_shuffled_vd = [vd[0] for vd in validation_data]
    en_shuffled_vd = [vd[1] for vd in validation_data]
    
    print("Baking batches...")
    train_batches = bake_batches(de_shuffled_td, en_shuffled_td, max_len=max_len_train)
    valid_batches = bake_batches(de_shuffled_vd, en_shuffled_vd, max_len=max_len_valid)
    print("Done.")
    
    start_time = time.time()
    
    train_loss = train(model, train_batches, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_batches, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Baking batches...
Done.
batch: 0 , 305.083392 MB used
batch: 100 , 1219.593728 MB used
Epoch: 01 | Time: 1m 23s
	Train Loss: 6.734 | Train PPL: 840.499
	 Val. Loss: 7.226 |  Val. PPL: 1375.229
Baking batches...
Done.
batch: 0 , 1214.748672 MB used
batch: 100 , 1219.72992 MB used
Epoch: 02 | Time: 1m 24s
	Train Loss: 5.903 | Train PPL: 366.096
	 Val. Loss: 7.114 |  Val. PPL: 1228.948
Baking batches...
Done.
batch: 0 , 1214.7456 MB used
batch: 100 , 1219.595776 MB used
Epoch: 03 | Time: 1m 25s
	Train Loss: 5.399 | Train PPL: 221.237
	 Val. Loss: 7.285 |  Val. PPL: 1458.989
Baking batches...
Done.
batch: 0 , 1214.743552 MB used
batch: 100 , 1219.593728 MB used
Epoch: 04 | Time: 1m 24s
	Train Loss: 4.907 | Train PPL: 135.293
	 Val. Loss: 7.175 |  Val. PPL: 1306.044
Baking batches...
Done.
batch: 0 , 1214.736896 MB used
batch: 100 , 1219.587072 MB used
Epoch: 05 | Time: 1m 24s
	Train Loss: 4.434 | Train PPL:  84.267
	 Val. Loss: 7.197 |  Val. PPL: 1334.802
Baking batches...
Done.
batch: 0 ,

In [0]:
results_test_a = {}
results_test_b = {}

results_test_a['train_loss'] = [4.951, 4.208, 3.919, 3.738, 3.631, 3.566, 3.511     ]
results_test_a['valid_loss'] = [5.028, 4.961, 5.049, 5.060, 5.087, 5.102, 5.142    ]
results_test_a['train_ppl'] = [141.283, 67.240, 50.341, 42.028, 37.761, 35.362, 33.467]
results_test_a['valid_ppl'] = [152.631, 142.753, 155.863, 157.552, 161.846, 164.395, 171.068]

results_test_b['train_loss'] = [4.945,  4.226, 3.940, 3.759, 3.651, 3.587, 3.536]
results_test_b['valid_loss'] = [5.310, 5.215, 5.187, 5.247, 5.323, 5.303, 5.368]
results_test_b['train_ppl'] = [140.527,  68.470, 51.401, 42.904, 38.516, 36.116, 34.334]
results_test_b['valid_ppl'] = [202.433, 183.977, 178.941, 189.977, 204.979, 200.840, 214.369]

In [0]:
def sample_net(net, batches):    
    preds = []
    targets = []
    sources = []
    sampled_batches = {}
       
    with torch.no_grad():
        for b in batches:
            source = b['source']
            target = b['target']
            
#             print("L length x N sentences:", target.shape)
        
            output = net.forward(source, target, teacher_forcing_ratio=0)
            
#             print("L length x N sentences x V vocabulary:", output.shape)
            
#             pred = torch.softmax(output, dim=2)

#             print("L length x N sentences x V vocabulary:", pred.shape)
            
            pred = torch.argmax(output, dim=2)
            
#             print("L length x N sentences:", pred.shape)
#             print("L length x N sentences:", target.shape)
            
            preds.append(pred)
            targets.append(target)
            sources.append(source)
    
    sampled_batches['source'] = sources
    sampled_batches['target'] = targets
    sampled_batches['prediction'] = preds
    
    return sampled_batches


batches = bake_batches(german['dev'], english['dev'])
sampled_batches = sample_net(model, batches)


In [0]:
def print_samples(batches):
    # Each of the inputs should be a tensor batch of shape L x N
    # print(sources.shape)

    source = batches['source']
    target = batches['target']
    prediction = batches['prediction']
    
    
    tgt_out = []
    pred_out = []
    for b in range(len(batches['source'])):
        for sent in range(source[b].shape[1]):
#             w = 1
#             idx = source[b][w,sent].item()
#             print("Source Sentence:")
#             while w < source[b][:,sent].shape[0] and idx!=3: 
#                 print(german['idx2word'][idx], end=" ")
#                 idx = source[b][w,sent].item()
#                 w+=1
            
            w = 1
            idx = target[b][w,sent].item()
            tgt_sent = []
#             print("\nTarget Sentence:")
            while w < target[b][:,sent].shape[0] and idx!=3:
#                 print(english['idx2word'][idx], end=" ")
                tgt_sent.append(english['idx2word'][idx])
                idx = target[b][w,sent].item()
                w+=1
            tgt_out.append(tgt_sent)


            w = 1
            idx = prediction[b][w,sent].item()
            pred_sent = []
#             print("\nPrediction:")
            while w < prediction[b][:,sent].shape[0] and idx!=3:
#                 print(english['idx2word'][idx], end=" ")
                pred_sent.append(english['idx2word'][idx])
                idx = prediction[b][w,sent].item()
                w+=1
            pred_out.append(pred_sent)

#             print("\n")
            
    with open('target.out', 'w') as f:
        for sent in tgt_out:
            x = " ".join(sent)
            f.write(x + '\n')
            
            
    with open('pred.out', 'w') as f:
        for sent in pred_out:
            y = " ".join(sent)
            f.write(y + '\n')

print_samples(sampled_batches)
    
# print(pred[0])
# print(target[0])



In [0]:
!git clone https://github.com/moses-smt/mosesdecoder.git
!pip install sacrebleu

fatal: destination path 'mosesdecoder' already exists and is not an empty directory.


In [0]:
%%shell
#!/bin/bash

# This is a reference to the gold translations from the dev set
REFERENCE_FILE="target.out"

# XXX: Change the following line to point to your model's output!
TRANSLATED_FILE="pred.out"

# The model output is expected to be in a tokenized form. Note, that if you
# tokenized your inputs to the model, then simply joined each output token with
# whitespace you should get tokenized outputs from your model.
# i.e. each output token is separate by whitespace
# e.g. "My model 's output is interesting ."
perl "mosesdecoder/scripts/tokenizer/detokenizer.perl" -l en < "$TRANSLATED_FILE" > "$TRANSLATED_FILE.detok"

PARAMS=("-tok" "intl" "-l" "de-en" "$REFERENCE_FILE")
sacrebleu "${PARAMS[@]}" < "$TRANSLATED_FILE.detok"

Detokenizer Version $Revision: 4134 $
Language: en
BLEU+case.mixed+lang.de-en+numrefs.1+smooth.exp+tok.intl+version.1.3.2 = 3.4 21.6/6.3/1.8/0.5 (BP = 1.000 ratio = 1.304 hyp_len = 14328 ref_len = 10991)
